In [1]:
#Import the Librabies

%matplotlib notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import plotly as py
import plotly.figure_factory as ff
import ipywidgets as widgets
import plotly.graph_objs as go
import pywt
import scipy.signal
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
py.offline.init_notebook_mode(connected = True)
from plotly import tools
from obspy.signal.detrend import polynomial

UPLOAD THE DATASET FILES PATH CONTAINS THE SIGNALS IN .pkl FILE. DATASET CONTAINS THE CSV FILE CONTAINS THE START TIME OF DIFFERENT ACTIVITY.

In [2]:
#Specify the path of PPG Field study dataset. It should have one.pkl file and activity file for start timings
# of individual activities.

path = "C:/Users/ee19s/Desktop/HR/PPG_FieldStudy/S2/S2.pkl"
dataset = pd.read_csv("C:/Users/ee19s/Desktop/HR/PPG_FieldStudy/S2/S2_activity.csv")

FETCH THE SIGNALS AND START TIME OF DIFFERENT ACTIVITIES

In [3]:
# Extract the ECG, accelerometer, reference respiratory signals.
with open (path , 'rb') as f:
    data_dict = pickle.load(f , encoding= 'bytes')

srate = 700
win_length = 32*srate
ecg_data = data_dict[b'signal'][b'chest'][b'ECG']
acc_data = data_dict[b'signal'][b'chest'][b'ACC']
respiratory_data = data_dict[b'signal'][b'chest'][b'Resp']
rpeaks = data_dict[b'rpeaks']
raw_ecg = np.array(ecg_data)
raw_respiratory = np.array(respiratory_data)
raw_ecg_data = raw_ecg.flatten()
raw_respiratory = raw_respiratory.flatten()
#Contains the start time of each activity.
start_time = dataset.iloc[: , 1].values

## DATA SEGMENTATION OF UNFILTERED RAW ECG SIGNAL

In [4]:
# Segment the raw ECG signal(unfiltered one) according to the activities.
baseline_ecg = raw_ecg_data[start_time[1]*srate : start_time[2] * srate]
stairs_ecg = raw_ecg_data[start_time[3]*srate : start_time[4] * srate]
soccer_ecg = raw_ecg_data[start_time[5]*srate : start_time[6] * srate]
cycling_ecg = raw_ecg_data[start_time[7]*srate : start_time[8] * srate]
driving_ecg = raw_ecg_data[start_time[9]*srate : start_time[10] * srate]
lunch_ecg = raw_ecg_data[start_time[11]*srate : start_time[12] * srate]
walking_ecg = raw_ecg_data[start_time[12]*srate : start_time[13] * srate]
working_ecg = raw_ecg_data[start_time[14]*srate : start_time[15] * srate]


In [5]:
#Reshape both filtered and unfiltered ecg data.
baseline_data = []
stairs_data = []
soccer_data = []
cycling_data = []
driving_data = []
lunch_data = []
walking_data = []
working_data = []

for i in range(len(baseline_ecg)//(win_length)):
    baseline_data.append(baseline_ecg[i*(win_length):(i+1)*(win_length)])
   
    
for i in range(len(stairs_ecg)//(win_length)):
    stairs_data.append(stairs_ecg[i*(win_length):(i+1)*(win_length)])
   
    
for i in range(len(soccer_ecg)//(win_length)):
    soccer_data.append(soccer_ecg[i*(win_length):(i+1)*(win_length)])
   
    
for i in range(len(cycling_ecg)//(win_length)):
    cycling_data.append(cycling_ecg[i*(win_length):(i+1)*(win_length)])
    
    
for i in range(len(driving_ecg)//(win_length)):
    driving_data.append(driving_ecg[i*(win_length):(i+1)*(win_length)])
    
for i in range(len(lunch_ecg)//(win_length)):
    lunch_data.append(lunch_ecg[i*(win_length):(i+1)*(win_length)])


for i in range(len(walking_ecg)//(win_length)):
    walking_data.append(walking_ecg[i*(win_length):(i+1)*(win_length)])

    
for i in range(len(working_ecg)//(win_length)):
    working_data.append(working_ecg[i*(win_length):(i+1)*(win_length)])


raw_baseline_ecg = np.array(baseline_data)
raw_stairs_ecg = np.array(stairs_data)
raw_soccer_ecg = np.array(soccer_data)
raw_cycling_ecg = np.array(cycling_data)
raw_driving_ecg = np.array(driving_data)
raw_lunch_ecg = np.array(lunch_data)
raw_walking_ecg = np.array(walking_data)
raw_working_ecg = np.array(working_data)

## ACCELEROMETER SIGNAL PREPROCESSING

In [6]:
acc_x = []
acc_y = []
acc_z = []
for item in acc_data:
    acc_x.append(item[0])
    acc_y.append(item[1])
    acc_z.append(item[2])

acc_x_axis = np.array(acc_x)
acc_y_axis = np.array(acc_y)
acc_z_axis = np.array(acc_z)

In [7]:
baseline_acc_x = acc_x_axis[start_time[1]*srate : start_time[2] * srate]
baseline_acc_y = acc_y_axis[start_time[1]*srate : start_time[2] * srate]
baseline_acc_z = acc_z_axis[start_time[1]*srate : start_time[2] * srate]

stairs_acc_x = acc_x_axis[start_time[3]*srate : start_time[4] * srate]
stairs_acc_y = acc_y_axis[start_time[3]*srate : start_time[4] * srate]
stairs_acc_z = acc_z_axis[start_time[3]*srate : start_time[4] * srate]

soccer_acc_x = acc_x_axis[start_time[5]*srate : start_time[6] * srate]
soccer_acc_y = acc_y_axis[start_time[5]*srate : start_time[6] * srate]
soccer_acc_z = acc_z_axis[start_time[5]*srate : start_time[6] * srate]

cycling_acc_x = acc_x_axis[start_time[7]*srate : start_time[8] * srate]
cycling_acc_y = acc_y_axis[start_time[7]*srate : start_time[8] * srate]
cycling_acc_z = acc_z_axis[start_time[7]*srate : start_time[8] * srate]

driving_acc_x = acc_x_axis[start_time[9]*srate : start_time[10] * srate]
driving_acc_y = acc_y_axis[start_time[9]*srate : start_time[10] * srate]
driving_acc_z = acc_z_axis[start_time[9]*srate : start_time[10] * srate]

lunch_acc_x = acc_x_axis[start_time[11]*srate : start_time[12] * srate]
lunch_acc_y = acc_y_axis[start_time[11]*srate : start_time[12] * srate]
lunch_acc_z = acc_z_axis[start_time[11]*srate : start_time[12] * srate]

walking_acc_x = acc_x_axis[start_time[12]*srate : start_time[13] * srate]
walking_acc_y = acc_y_axis[start_time[12]*srate : start_time[13] * srate]
walking_acc_z = acc_z_axis[start_time[12]*srate : start_time[13] * srate]

working_acc_x = acc_x_axis[start_time[14]*srate : start_time[15] * srate]
working_acc_y = acc_y_axis[start_time[14]*srate : start_time[15] * srate]
working_acc_z = acc_z_axis[start_time[14]*srate : start_time[15] * srate]

In [8]:
# Reshape the accelerometer and respiratory signal obtained from accelerometer.
baseline_data_x = []
baseline_data_y = []
baseline_data_z = []


stairs_data_x = []
stairs_data_y = []
stairs_data_z = []


soccer_data_x = []
soccer_data_y = []
soccer_data_z = []


cycling_data_x = []
cycling_data_y = []
cycling_data_z = []


driving_data_x = []
driving_data_y = []
driving_data_z = []


lunch_data_x = []
lunch_data_y = []
lunch_data_z = []


walking_data_x = []
walking_data_y = []
walking_data_z = []


working_data_x = []
working_data_y = []
working_data_z = []


for i in range(len(baseline_acc_x)//(win_length)):
    baseline_data_x.append(baseline_acc_x[i*(win_length):(i+1)*(win_length)])
    baseline_data_y.append(baseline_acc_y[i*(win_length):(i+1)*(win_length)])
    baseline_data_z.append(baseline_acc_z[i*(win_length):(i+1)*(win_length)])
   
    
for i in range(len(stairs_acc_x)//(win_length)):
    stairs_data_x.append(stairs_acc_x[i*(win_length):(i+1)*(win_length)])
    stairs_data_y.append(stairs_acc_y[i*(win_length):(i+1)*(win_length)])
    stairs_data_z.append(stairs_acc_y[i*(win_length):(i+1)*(win_length)])
    

for i in range(len(soccer_acc_x)//(win_length)):
    soccer_data_x.append(soccer_acc_x[i*(win_length):(i+1)*(win_length)])
    soccer_data_y.append(soccer_acc_y[i*(win_length):(i+1)*(win_length)])
    soccer_data_z.append(soccer_acc_z[i*(win_length):(i+1)*(win_length)])
    
    
for i in range(len(cycling_acc_x)//(win_length)):
    cycling_data_x.append(cycling_acc_x[i*(win_length):(i+1)*(win_length)])
    cycling_data_y.append(cycling_acc_y[i*(win_length):(i+1)*(win_length)])
    cycling_data_z.append(cycling_acc_z[i*(win_length):(i+1)*(win_length)])
  
    
for i in range(len(driving_acc_x)//(win_length)):
    driving_data_x.append(driving_acc_x[i*(win_length):(i+1)*(win_length)])
    driving_data_y.append(driving_acc_y[i*(win_length):(i+1)*(win_length)])
    driving_data_z.append(driving_acc_z[i*(win_length):(i+1)*(win_length)])

    
for i in range(len(lunch_acc_x)//(win_length)):
    lunch_data_x.append(lunch_acc_x[i*(win_length):(i+1)*(win_length)])
    lunch_data_y.append(lunch_acc_y[i*(win_length):(i+1)*(win_length)])
    lunch_data_z.append(lunch_acc_z[i*(win_length):(i+1)*(win_length)])
    
    
for i in range(len(walking_acc_x)//(win_length)):
    walking_data_x.append(walking_acc_x[i*(win_length):(i+1)*(win_length)])
    walking_data_y.append(walking_acc_y[i*(win_length):(i+1)*(win_length)])
    walking_data_z.append(walking_acc_z[i*(win_length):(i+1)*(win_length)])
    
    
for i in range(len(working_acc_x)//(win_length)):
    working_data_x.append(working_acc_x[i*(win_length):(i+1)*(win_length)])
    working_data_y.append(working_acc_y[i*(win_length):(i+1)*(win_length)])
    working_data_z.append(working_acc_z[i*(win_length):(i+1)*(win_length)])
   
    
raw_baseline_acc_x = np.array(baseline_data_x)
raw_baseline_acc_y = np.array(baseline_data_y)
raw_baseline_acc_z = np.array(baseline_data_z)


raw_stairs_acc_x = np.array(stairs_data_x)
raw_stairs_acc_y = np.array(stairs_data_y)
raw_stairs_acc_z = np.array(stairs_data_z)


raw_soccer_acc_x = np.array(soccer_data_x)
raw_soccer_acc_y = np.array(soccer_data_y)
raw_soccer_acc_z = np.array(soccer_data_z)


raw_cycling_acc_x = np.array(cycling_data_x)
raw_cycling_acc_y = np.array(cycling_data_y)
raw_cycling_acc_z = np.array(cycling_data_z)


raw_driving_acc_x = np.array(driving_data_x)
raw_driving_acc_y = np.array(driving_data_y)
raw_driving_acc_z = np.array(driving_data_z)


raw_lunch_acc_x = np.array(lunch_data_x)
raw_lunch_acc_y = np.array(lunch_data_y)
raw_lunch_acc_z = np.array(lunch_data_z)


raw_walking_acc_x = np.array(walking_data_x)
raw_walking_acc_y = np.array(walking_data_y)
raw_walking_acc_z = np.array(walking_data_z)


raw_working_acc_x = np.array(working_data_x)
raw_working_acc_y = np.array(working_data_y)
raw_working_acc_z = np.array(working_data_z)


## PREPROCESSING OF REFERENCE RESPIRATORY SIGNAL

In [9]:
# FILTERING OF RESPIRATORY SIGNAL TO BE IN RANGE OF RESPIRATORY FREQUENCY.
flp_respB , flp_respA = scipy.signal.cheby2(6,20,0.6/(srate/2) , btype='lowpass')
fhp_respB , fhp_respA = scipy.signal.cheby2(4,20, 0.1/(srate/2) , btype='highpass')
lp_filt_resp_sig = scipy.signal.filtfilt(flp_respB , flp_respA , raw_respiratory)
processed_resp_sig = scipy.signal.filtfilt(fhp_respB,fhp_respA , lp_filt_resp_sig)


In [10]:
# Segment the filtered respiratory signal according to the activity.
proc_baseline_resp = processed_resp_sig[start_time[1]*srate : start_time[2] * srate]
proc_stairs_resp = processed_resp_sig[start_time[3]*srate : start_time[4] * srate]
proc_soccer_resp = processed_resp_sig[start_time[5]*srate : start_time[6] * srate]
proc_cycling_resp = processed_resp_sig[start_time[7]*srate : start_time[8] * srate]
proc_driving_resp = processed_resp_sig[start_time[9]*srate : start_time[10] * srate]
proc_lunch_resp = processed_resp_sig[start_time[11]*srate : start_time[12] * srate]
proc_walking_resp = processed_resp_sig[start_time[12]*srate : start_time[13] * srate]
proc_working_resp = processed_resp_sig[start_time[14]*srate : start_time[15] * srate]

In [11]:
proc_baseline_data_resp = []
proc_stairs_data_resp = []
proc_soccer_data_resp = []
proc_cycling_data_resp = []
proc_driving_data_resp = []
proc_lunch_data_resp = []
proc_walking_data_resp = []
proc_working_data_resp = []

for i in range(len(proc_baseline_resp)//(win_length)):
    proc_baseline_data_resp.append(proc_baseline_resp[i*(win_length):(i+1)*(win_length)])
    
for i in range(len(proc_stairs_resp)//(win_length)):
    proc_stairs_data_resp.append(proc_stairs_resp[i*(win_length):(i+1)*(win_length)])
    
for i in range(len(proc_soccer_resp)//(win_length)):
    proc_soccer_data_resp.append(proc_soccer_resp[i*(win_length):(i+1)*(win_length)])
    
for i in range(len(proc_cycling_resp)//(win_length)):
    proc_cycling_data_resp.append(proc_cycling_resp[i*(win_length):(i+1)*(win_length)])
    
for i in range(len(proc_driving_resp)//(win_length)):
    proc_driving_data_resp.append(proc_driving_resp[i*(win_length):(i+1)*(win_length)])
    
for i in range(len(proc_lunch_resp)//(win_length)):
    proc_lunch_data_resp.append(proc_lunch_resp[i*(win_length):(i+1)*(win_length)])

for i in range(len(proc_walking_resp)//(win_length)):
    proc_walking_data_resp.append(proc_walking_resp[i*(win_length):(i+1)*(win_length)])
    
for i in range(len(proc_working_resp)//(win_length)):
    proc_working_data_resp.append(proc_working_resp[i*(win_length):(i+1)*(win_length)])
    
raw_baseline_resp_proc = np.array(proc_baseline_data_resp)
raw_stairs_resp_proc = np.array(proc_stairs_data_resp)
raw_soccer_resp_proc = np.array(proc_soccer_data_resp)
raw_cycling_resp_proc = np.array(proc_cycling_data_resp)
raw_driving_resp_proc = np.array(proc_driving_data_resp)
raw_lunch_resp_proc = np.array(proc_lunch_data_resp)
raw_walking_resp_proc = np.array(proc_walking_data_resp)
raw_working_resp_proc = np.array(proc_working_data_resp)

## SET THE WIDGETS ACCORDING TO CHOOSE DIFFERENT ACTIVITIES

In [12]:
drop_down = widgets.Dropdown(options=['baseline','stairs','soccer','cycling','driving','lunch','walking','working'],
                                description='activity',
                                disabled=False)
display(drop_down)

Dropdown(description='activity', options=('baseline', 'stairs', 'soccer', 'cycling', 'driving', 'lunch', 'walk…

In [13]:
drop_down.value

'baseline'

## DISTRIBUTE THE SIGNALS ACCORDING TO THE ACTIVITY.

In [16]:

if drop_down.value == 'baseline':
    total_windows_ecg = len(baseline_ecg)//(win_length)
    total_windows_acc = len(baseline_acc_x)//(win_length)
    total_windows_resp = len(proc_baseline_resp)//(win_length)
    patient_ecg = raw_baseline_ecg
    patient_acc_x = raw_baseline_acc_x     
    patient_acc_y = raw_baseline_acc_y
    patient_acc_z = raw_baseline_acc_z
    patient_resp_proc = raw_baseline_resp_proc
       
if drop_down.value == 'stairs':
    total_windows_ecg = len(stairs_ecg)//(win_length)
    total_windows_acc = len(stairs_acc_x)//(win_length)
    total_windows_resp = len(proc_stairs_resp)//(win_length)
    patient_ecg = raw_stairs_ecg
    patient_acc_x = raw_stairs_acc_x     
    patient_acc_y = raw_stairs_acc_y 
    patient_acc_z = raw_stairs_acc_z
    patient_resp_proc = raw_stairs_resp_proc
       
if drop_down.value == 'soccer':
    total_windows_ecg = len(soccer_ecg)//(win_length)
    total_windows_acc = len(soccer_acc_x)//(win_length)
    total_windows_resp = len(proc_soccer_resp)//(win_length) 
    patient_ecg = raw_soccer_ecg
    patient_acc_x = raw_soccer_acc_x     
    patient_acc_y = raw_soccer_acc_y 
    patient_acc_z = raw_soccer_acc_z
    patient_resp_proc = raw_soccer_resp_proc
       
if drop_down.value == 'cycling':
    total_windows_ecg = len(cycling_ecg)//(win_length)
    total_windows_acc = len(cycling_acc_x)//(win_length)
    total_windows_resp = len(proc_cycling_resp)//(win_length)
    patient_ecg = raw_cycling_ecg
    patient_acc_x = raw_cycling_acc_x     
    patient_acc_y = raw_cycling_acc_y 
    patient_acc_z = raw_cycling_acc_z
    patient_resp_proc = raw_cycling_resp_proc
      
if drop_down.value == 'driving':
    total_windows_ecg = len(driving_ecg)//(win_length)
    total_windows_acc = len(driving_acc_x)//(win_length)
    total_windows_resp = len(proc_driving_resp)//(win_length)
    patient_ecg = raw_driving_ecg
    patient_acc_x = raw_driving_acc_x     
    patient_acc_y = raw_driving_acc_y 
    patient_acc_z = raw_driving_acc_z    
    patient_resp_proc = raw_driving_resp_proc

if drop_down.value == 'lunch':
    total_windows_ecg = len(lunch_ecg)//(win_length)
    total_windows_acc = len(lunch_acc_x)//(win_length)
    total_windows_resp = len(proc_lunch_resp)//(win_length)
    patient_ecg = raw_lunch_ecg
    patient_acc_x = raw_lunch_acc_x     
    patient_acc_y = raw_lunch_acc_y 
    patient_acc_z = raw_lunch_acc_z
    patient_resp_proc = raw_lunch_resp_proc
    
if drop_down.value == 'walking':
    total_windows_ecg = len(walking_ecg)//(win_length)
    total_windows_acc = len(walking_acc_x)//(win_length)
    total_windows_resp = len(proc_walking_resp)//(win_length)
    patient_ecg = raw_walking_ecg
    patient_acc_x = raw_walking_acc_x     
    patient_acc_y = raw_walking_acc_y 
    patient_acc_z = raw_walking_acc_z    
    patient_resp_proc = raw_walking_resp_proc

if drop_down.value == 'working':
    total_windows_ecg = len(working_ecg)//(win_length)
    total_windows_acc = len(working_acc_x)//(win_length)
    total_windows_resp = len(proc_working_resp)//(win_length)
    patient_ecg = raw_working_ecg
    patient_acc_x = raw_working_acc_x     
    patient_acc_y = raw_working_acc_y 
    patient_acc_z = raw_working_acc_z   
    patient_resp_proc = raw_working_resp_proc


## FRAME THE LAYOUTS AND OBSERVE THE PLOTS

In [17]:
# Frame the sampling axis
no_of_samples_in_win = win_length # for reference signals mostley.
x = np.linspace(start = 0,stop = no_of_samples_in_win, num = no_of_samples_in_win) 

# Layout for refrence ECG and Processed ECG
layout_ecg = go.Layout(
    title = "Plotting ECG Waveforms for "+ drop_down.value,
    yaxis = dict(
         title = 'Amplitude' 
    ),
    xaxis = dict(
        title = 'samples per minute'
    )
)

# Layout for raw x-axis accelerometer data.
layout_x = go.Layout(
    title = 'Plotting X-axis Accelerometer Waveforms for '+ drop_down.value ,
    yaxis = dict(
         title = 'Amplitude' 
    ),
    xaxis = dict(
        title = 'samples per minute'
    )
)

# Layout for raw y-axis accelerometer data.
layout_y = go.Layout(
    title = 'Plotting Y-axis Accelerometer Waveforms for '+ drop_down.value ,
    yaxis = dict(
         title = 'Amplitude' 
    ),
    xaxis = dict(
        title = 'samples per minute'
    )
)

# Layout for raw z-axis accelerometer data.
layout_z = go.Layout(
    title = 'Plotting Z-axis Accelerometer Waveforms for ' + drop_down.value ,
    yaxis = dict(
         title = 'Amplitude' 
    ),
    xaxis = dict(
        title = 'samples per minute'
    )
)

# Layout for reference respiration signal.
layout_resp = go.Layout(
    title = 'Plotting Respiratory Waveforms for '+ drop_down.value,
    yaxis = dict(
         title = 'Amplitude' 
    ),
    xaxis = dict(
        title = 'samples per minute'
    )
)

def update_plot(signals):
    data = []
    # Reference ECG trace
    trace_ecg = go.Scatter(
             x = x,
             y = patient_ecg[signals,:], 
             mode = 'lines',
             name = 'ecg_plot_raw{}'.format(signals),
             line = dict(
                 shape = 'spline'
             )
        )

    # Reference Accelerometer signal x-axis trace
    tracex_acc = go.Scatter(
             x = x,
             y = patient_acc_x[signals,:], 
             mode = 'lines',
             name = 'acc_plot{}'.format(signals),
             line = dict(
                 shape = 'spline'
             )
        )
    # Reference Accelerometer signal y-axis trace
    tracey_acc = go.Scatter(
             x = x,
             y = patient_acc_y[signals,:], 
             mode = 'lines',
             name = 'acc_plot{}'.format(signals),
             line = dict(
                 shape = 'spline'
             )
        )
    # Reference Accelerometer signal z-axis trace
    tracez_acc = go.Scatter(
             x = x,
             y = patient_acc_z[signals,:], 
             mode = 'lines',
             name = 'acc_plot{}'.format(signals),
             line = dict(
                 shape = 'spline'
             )
        )

    # Reference Respiratory signal trace after pre processing.
    trace_resp_proc = go.Scatter(
             x = x,
             y = patient_resp_proc[signals,:], 
             mode = 'lines',
             name = 'ref_resp_proc{}'.format(signals),
             line = dict(
                 shape = 'spline'
             )
        )

    fig_ecg = go.Figure(data = [trace_ecg],layout = layout_ecg)
    py.offline.iplot(fig_ecg)
    figx = go.Figure(data = [tracex_acc],layout = layout_x)
    py.offline.iplot(figx)
    figy = go.Figure(data = [tracey_acc],layout = layout_y)
    py.offline.iplot(figy)
    figz = go.Figure(data = [tracez_acc],layout = layout_z)
    py.offline.iplot(figz)
    fig_resp = go.Figure(data = [trace_resp_proc],layout = layout_resp)
    py.offline.iplot(fig_resp)

        
signals_ecg = widgets.IntSlider(min = 0,max = total_windows_ecg, value = 0, description = 'Record_no:')
widgets.interactive(update_plot, signals = signals_ecg)

signals_acc = widgets.IntSlider(min = 0,max = total_windows_acc, value = 0, description = 'Record_no:')
widgets.interactive(update_plot, signals = signals_acc)

signals_resp = widgets.IntSlider(min = 0,max = total_windows_resp, value = 0, description = 'Record_no:')
widgets.interactive(update_plot, signals = signals_resp)




interactive(children=(IntSlider(value=0, description='Record_no:', max=12), Output()), _dom_classes=('widget-i…